In [1]:
import pip 
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import subprocess
subprocess.run(['pip', 'install', '-r', 'requirements.txt'])
import nest_asyncio
nest_asyncio.apply()
import asyncio
import vl_convert as vlc
import nltk
import pandas as pd
import csv
import re
import toolz
import io
import os
import altair_saver
import lyricsgenius
import requests
import time
import altair as alt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
nltk.download('wordnet')
from lyricsgenius import Genius
import numpy as np
from dotenv import load_dotenv
import discord 
from discord.ext import commands
from requests.exceptions import HTTPError, Timeout

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
load_dotenv('3510.env')

True

In [3]:
Token = os.getenv('DISCORD_TOKEN')
spotifyClient = os.getenv('CLIENT_ID')
spotifySecret = os.getenv('CLIENT_SECRET')
lyricsGenius = os.getenv('LYRICS_GENIUS')
intents = discord.Intents().all()

In [4]:
try:
    Genius(Token)
except HTTPError as e:
    print(e.errno)    # status code
    print(e.args[0])  # status code
    print(e.args[1])  # error message
except Timeout:
    pass

In [5]:
# Importing necessary libraries
from nltk.stem import WordNetLemmatizer
import re
import lyricsgenius
from textblob import TextBlob

# Initializing WordNetLemmatizer for lemmatization
lemmatizer = WordNetLemmatizer()


# Creating a Genius API object with the obtained API key
genius = lyricsgenius.Genius(lyricsGenius)

# Function to process sentiment polarity of song lyrics
def process_title(title, artist):
    # Check if title or artist is empty or contains only whitespace
    if not title.strip() or not artist.strip():
        return None  # If either title or artist is empty or contains only whitespace, return None
    
    # Search for the song using the provided title and artist
    song = genius.search_song(title, artist)
    
    # If the song is found
    if song:
        lyrics = song.lyrics  # Get the lyrics of the song
        
        # Perform text preprocessing (replace newlines and question marks, lemmatization)
        lemmatized_lyrics = ' '.join([lemmatizer.lemmatize(word) for word in re.sub(r"\n{1,2}", ". ", re.sub(r"\?\n{1,2}", "? ", lyrics)).split()])
        
        # Calculate sentiment polarity using TextBlob sentiment analysis on the preprocessed lyrics
        sentiment_polarity = TextBlob(lemmatized_lyrics).sentiment.polarity
        
        return sentiment_polarity  # Return the sentiment polarity of the lyrics
    else:
        return None  # Return None if the song is not found


In [6]:
def process_subj(title, artist):
    # Check if title or artist is empty or contains only whitespace
    if not title.strip() or not artist.strip():
        return None  # If either title or artist is empty or contains only whitespace, return None
    
    # Search for the song using the provided title and artist
    song = genius.search_song(title, artist)
    
    # If the song is found
    if song:
        lyrics = song.lyrics  # Get the lyrics of the song
        
        # Perform text preprocessing (replace newlines and question marks, lemmatization)
        lemmatized_lyrics = ' '.join([lemmatizer.lemmatize(word) for word in re.sub(r"\n{1,2}", ". ", re.sub(r"\?\n{1,2}", "? ", lyrics)).split()])
        
        # Calculate subjectivity using TextBlob sentiment analysis on the preprocessed lyrics
        subjectivity = TextBlob(lemmatized_lyrics).sentiment.subjectivity
        
        return subjectivity  # Return the subjectivity score of the lyrics
    else:
        return None  # Return None if the song is not found



In [7]:
def fetch_lyrics(title, artist):
    # Check if title or artist is empty or contains only whitespace
    if not title.strip() or not artist.strip():
        return None  # If either title or artist is empty or contains only whitespace, return None
    
    # Search for the song using the provided title and artist
    song = genius.search_song(title, artist)
    
    # If the song is found
    if song:
        lyrics = song.lyrics  # Get the lyrics of the song
        return lyrics  # Return the lyrics of the song


In [8]:
def lyrics_finder(lyrics):
    # Search for all content related to the provided lyrics (assuming through an external API like Genius)
    request = genius.search_all(lyrics)
    
    # Iterate through hits in the third section of the search results (assuming it's the section related to songs)
    for hit in request['sections'][2]['hits']:
        # Return the title of the song and the name of the primary artist
        return hit['result']['title'] + ' by ' + hit['result']['primary_artist']['name']


In [9]:
def album_gets(album, artist):
    # Construct a search query combining the album and artist names
    search_query = f'{album} {artist}'
    
    # Search for albums using the constructed search query (assuming this uses an external API like Genius)
    album_search = genius.search_albums(search_query, per_page=1)
    
    # Check if there are any sections and hits in the search results
    if 'sections' in album_search and album_search['sections']:
        first_section = album_search['sections'][0]
        hits = first_section.get('hits', [])
        
        # If there are hits, extract details about the first hit (assuming it's the most relevant)
        if hits:
            first_hit = hits[0]
            result = first_hit.get('result', {})
            album_id = result.get('id')
            album_cover = result.get('cover_art_url')  # Extract the album cover URL
    
    # Retrieve tracks from the album using its ID obtained above
    album_tracks = genius.album_tracks(album_id, per_page=50)
    
    songs = []
    # If tracks are available in the album tracks response, extract the song titles
    if 'tracks' in album_tracks:
        for track in album_tracks['tracks']:
            song_title = track.get('song', {}).get('title')
            songs.append(song_title)
    
    # If no songs were found, return an empty list and None for album cover; otherwise, return the songs and album cover
    if songs == []:
        return [], None 
    return songs, album_cover



In [10]:
import pandas as pd
import altair as alt

def visualizer(songs, artist):
    # Initialize empty lists to store sentiment and subjectivity scores
    sentScores = []
    subjScores = []
    
    # Iterate through each song to process sentiment and subjectivity
    for song in songs:
        # Process sentiment and append to the list
        sentScores.append(process_title(song, artist))
        
        # Process subjectivity and append to the list
        subjScores.append(process_subj(song, artist))
                          
    # Create a DataFrame with song titles, sentiment, and subjectivity scores
    data = pd.DataFrame({
        'song_title': songs,
        'sentiment': sentScores,
        'subjectivity': subjScores})
    
    # Calculate the average sentiment and subjectivity scores for the album
    avg_sent_score = data['sentiment'].mean()
    avg_subj_score = data['subjectivity'].mean()
    
    # Generate visualization using Altair
    visual = alt.Chart(data).mark_rect().encode(
        x=alt.X("song_title:N"),  # X-axis: Song titles
        y=alt.Y("sentiment:Q", title="Sentiment Score"),  # Y-axis: Sentiment scores
        color=alt.Color("subjectivity:Q"),  # Color-coded by subjectivity scores
    ).properties(
        title="Album visualization of sentiment and subjectivity of songs"  # Chart title
    )
    
    # Return DataFrame, visualization, and average scores
    return data, visual, avg_sent_score, avg_subj_score

In [11]:
client_credentials_manager = SpotifyClientCredentials(client_id=spotifyClient,client_secret=spotifySecret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Function to get song recommendations based on an inputted song and artist
def get_song_recommendations(track_name, artist_name):
    # Search for the track and artist
    query = f"track:{track_name} artist:{artist_name}"
    results = sp.search(q=query, limit=1, type='track')

    # Check if tracks are found
    if results['tracks']['items']:
        # Get the track ID of the first track found
        track_id = results['tracks']['items'][0]['id']

        # Get recommendations based on the found track
        recommendations = sp.recommendations(seed_tracks=[track_id], limit=10)
        
        # Extract track names from recommendations
        recommended_tracks = [track['name'] for track in recommendations['tracks']]
        return recommended_tracks
    
    return None  # Return None if the track is not found

In [12]:
client_credentials_manager = SpotifyClientCredentials(client_id=spotifyClient, client_secret=spotifySecret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
def get_artist_recommendations(artist_name):
    # Search for the artist
    results = sp.search(q=f"artist:{artist_name}", limit=1, type='artist')

    # Check if artists are found
    if results['artists']['items']:
        # Get the artist ID of the first artist found
        artist_id = results['artists']['items'][0]['id']

        # Get recommendations based on the found artist
        recommendations = sp.recommendations(seed_artists=[artist_id], limit=10)
        
        # Extract unique artist names from recommendations, excluding the inputted artist
        recommended_artists = set()
        for track in recommendations['tracks']:
            for artist in track['artists']:
                if artist['name'].lower() != artist_name.lower():
                    recommended_artists.add(artist['name'])

        # Convert the set to a list for consistent ordering
        recommended_artists = list(recommended_artists)

        return recommended_artists[:10]  # Return the first 10 unique recommended artists
    
    return None

In [13]:
def get_album_recommendations(album_name, artist_name):
    # Search for the artist
    results = sp.search(q=f"artist:{artist_name}", limit=1, type='artist')

    # Check if artists are found
    if results['artists']['items']:
        # Get the artist ID of the first artist found
        artist_id = results['artists']['items'][0]['id']

        # Search for the album within the artist's discography
        albums = sp.artist_albums(artist_id, album_type='album', limit=50)['items']
        album_ids = [album['id'] for album in albums if album['name'].lower() == album_name.lower()]

        if album_ids:
            print(f"Found album ID: {album_ids[0]}")

            # Get the first track from the album
            album_tracks = sp.album_tracks(album_ids[0])['items']
            if album_tracks:
                # Use the first track as a seed for recommendations
                track_id = album_tracks[0]['id']
                
                # Get recommendations based on the found track ID
                recommendations = sp.recommendations(seed_tracks=[track_id], limit=10)

                # Extract unique album names from recommendations
                recommended_albums = set()
                for track in recommendations['tracks']:
                    for album in track['album']['artists']:
                        recommended_albums.add(album['name'] + " - " + track['album']['name'])

                # Convert the set to a list for consistent ordering
                recommended_albums = list(recommended_albums)

                return recommended_albums[:10]  # Return the first 10 unique recommended albums

    print(f"Artist or album not found. Artist: {artist_name}, Album: {album_name}")
    return None  # Return None if the artist or album is not found


In [14]:
bot = commands.Bot(command_prefix='!', intents = intents)

In [15]:
time.sleep(30)

In [ ]:
# Event triggered when the bot is ready
@bot.event
async def on_ready():
    print('{0} has connected to Discord.'.format(bot.user.id, bot.user.name))
    # Sending a message to each text channel the bot has access to upon joining a server
    for guild in bot.guilds:
        for channel in guild.text_channels:
            if channel.permissions_for(guild.me).send_messages:
                await channel.send("Bot has joined the server!")
                await channel.send('Please type !start to get started')
                break
#  Command to provide instructions for using the bot
@bot.command(name="start", help="Instructions for bot")
async def rogan(ctx):
    message = (
        # Instructions and guidelines for using various commands
        'This discord bot has an array of commands please use freely\n'
        'If servers time out please rerun\n'
        'Inputs are case sensitive so make sure you enter your inputs properly\n'
        'Please do not use multiple commands at the same time\n'
        'Some Commands may take a few minutes to fully send to discord\n'
        'lyricGet command only works with puncuation and proper spelling, have not figured out how to make it not case sensitive\n'
        'The commands for this bot are:\n'
        '`!songSubj`: This gives a song subjectivity score\n'
        '`!songSent`: Gives a song sentiment score\n'
        '`!songLyrics`: Prints song lyrics\n'
        '`!lyricGet`: gets song based of inputed lyrics and artist\n'
        '`!albumGet`: gets songs from an inputed album\n'
        '`!albumGetProp`: gets songs from an inputed album but includes properties and visualization\n'
        '`!albumGetCover`: gets album cover from input\n'
        '`!songRec`: gets recommended songs based off an inputted song and artist\n'
        '`!artistRec`: gets recommended Artists based off an inputted artist\n'
        '`!albumRec`: gets recommended albums based off an inputted album and artist\n'
        'When using commands !songsubj, !songsent, !songlyrics, write the song name, then add a comma, and then the artist name\n'
        'When using commands !lyricget ,write the lyrics, remember it is caseSensitve and use proper punctuation\n'
        'When using commands !albumGet, !albumGetProp, !albumGetCover, write the Album name, then add a comma, and then the artist name\n'
        'Example !albumGetProp Graduation, Kanye West\n'
        'Example !songLyrics Eleanor Rigby, Beatles\n'
       
    )
    await ctx.send(message)
# Command to get a list of songs from an album           
@bot.command(name="albumGet", help="gets list of songs from an album")
async def songlook(ctx, *, song_info):
# Splitting the input into album and artist
    split_info = song_info.split(',', 1)
    if len(split_info) != 2:
        await ctx.send("Please use the format: Album Name, Artist Name")
        return

    album, artist = map(str.strip, split_info)
    songs,album_cover = album_gets(album, artist)
    if not songs:
        await ctx.send("Album not found.")
        return
    await ctx.send('The Album ' + album + ' include these tracks by ' + artist)
    for song in songs:
        if song is not None:
            await ctx.send(f"{song}")
        else:
            await ctx.send(f"Song names for '{album}' by {artist} are unavailable.")
# Command to get album information with sentiment, subjectivity scores, and visualization            
@bot.command(name="albumGetProp", help="gets list of songs from an album with their sentiment, subjectivity scores, album cover, and viusalization ")
async def songlook(ctx, *, song_info):
    # Splitting the input into album and artist
    split_info = song_info.split(',', 1)
    if len(split_info) != 2:
        await ctx.send("Please use the format: Album Name, Artist Name")
        return
    await ctx.send("Processing may take a few minutes")
    
    album, artist = map(str.strip, split_info)
    songs,album_cover = album_gets(album, artist)
    if not songs:
        await ctx.send("Album not found.")
        return
    # Fetching data and visualization for the album
    data, visual ,avg_sent_score, avg_subj_score = visualizer(songs, artist)
    if not data.empty:
        formatted_data = data.to_string(index=True)  
        await ctx.send(f"Here's the data:\n```{formatted_data}```")
        await ctx.send(f"The average Sentiment Score of this album is: {round(avg_sent_score,2)}\n and the average subjectivity score of this album is: {round(avg_sent_score,2)}\n")
        await ctx.send(f"Album Cover:\n{album_cover}")
    else:
        await ctx.send(f"Album Properties for '{album}' by {artist} are unavailable.")
    # Sending album properties and visualization to the Discord channel
    # ...
    if visual:
        png_data = vlc.vegalite_to_png(visual.to_json(), scale=2)
        with open("chart.png", "wb") as f:
            f.write(png_data)
        file_obj = discord.File("chart.png", filename="chart.png")
        await ctx.send("Album Visualization", file=file_obj)
# Command to get the album cover for an inputted album     
@bot.command(name="albumGetCover", help="gets Album Cover for inputed album")
async def songlook(ctx, *, song_info):
    # Splitting the input into album and artist
    split_info = song_info.split(',', 1)
    if len(split_info) != 2:
        await ctx.send("Please use the format: Album Name, Artist Name")
        return
    album, artist = map(str.strip, split_info)
    songs,album_cover = album_gets(album, artist)
    if not songs:
        await ctx.send("Album not found.")
        return
    if songs:
        await ctx.send(f"Album Cover:\n{album_cover}")
    else:
        await ctx.send(f"Album cover for '{album}' by {artist} are unavailable.")
# Command to fetch sentiment score of a song    
@bot.command(name="songSent", help="fetch song print sentiment")
async def songlook(ctx, *, song_info):
# Splitting the input into song title and artist
    split_info = song_info.split(',', 1)
    if len(split_info) != 2:
        await ctx.send("Please use the format: Album Name, Artist Name")
        return
    # Fetching sentiment of the song and sending it to the Discord channel
    song_title, artist_name = map(str.strip, split_info)
    
    sent = process_title(song_title, artist_name)
    
    if sent is not None:
        await ctx.send(f"Sentiment polarity of '{song_title}' by {artist_name}: {round(sent,2)}")
    else:
        await ctx.send("Song not found or lyrics unavailable.")
# Command to fetch subjectivity score of a song     
@bot.command(name="songSubj", help="fetch song print subjectivity")
async def songsubj(ctx, *, song_info):
    # Splitting the input into song title and artist
    split_info = song_info.split(',', 1)
    if len(split_info) != 2:
        await ctx.send("Please use the format: Song Title, Artist Name")
        return
    # Fetching subjectivity of the song and sending it to the Discord channel
    song_title, artist_name = map(str.strip, split_info)
    
    subj = process_subj(song_title, artist_name)
    
    if subj is not None:
        await ctx.send(f"Subjectivity polarity of '{song_title}' by {artist_name}: {round(subj,2)}")
    else:
        await ctx.send("Song not found or lyrics unavailable.")

# Command to fetch lyrics of a song
@bot.command(name="songLyrics", help="fetch song print lyrics")
async def songlyrics(ctx, *, song_info):
    # Splitting the input into song title and artist
    split_info = song_info.split(',', 1)
    if len(split_info) != 2:
        await ctx.send("Please use the format: Song Title, Artist Name")
        return
    # Fetching lyrics of the song and sending it to the Discord channel
    song_title, artist_name = map(str.strip, split_info)
    
    lyrics = fetch_lyrics(song_title, artist_name)
    
    if lyrics:
        chunks = lyrics.split('\n\n')  
        
        for chunk in chunks:
            await ctx.send(f"{song_title}, {artist_name}:\n{chunk}")
    else:
        await ctx.send("Song not found or lyrics unavailable.")
# Command to find a song containing inputed lyrics       
@bot.command(name="lyricGet", help="Searches for a song containing inputed lyrics")
async def getsong(ctx, *, song_info):
    # Finding a song based on inputed lyrics
    get = lyrics_finder(song_info)
    print(f"Returned value from lyrics_finder(): {get}")  
    # Sending the name of the song (if found) to the Discord channel
    if get:  
        await ctx.send(f"Song name is '{get}'")
    else:
        await ctx.send("Song not found or lyrics unavailable.")

# Running the bot with the provided token
@bot.command(name="songRec", help="gets song recommendations based off inputed song and artist")
async def songlook(ctx, *, song_info):
# Splitting the input into album and artist
    split_info = song_info.split(',', 1)
    if len(split_info) != 2:
        await ctx.send("Please use the format: song name, Artist Name")
        return

    song, artist = map(str.strip, split_info)
    recommendations = get_song_recommendations(song, artist)
    if recommendations:
        await ctx.send(f"Recommended tracks based on '{song}' by '{artist}':")
        for idx, song in enumerate(recommendations, 1):
            await ctx.send(f"{idx}. {song}")
    else:
        await ctx.send("Track not found or recommendations unavailable.")
        
@bot.command(name="artistRec", help="gets artist recommendations based off inputed artist")
async def songlook(ctx, *, artist_info):
    recommendations = get_artist_recommendations(artist_info)
    if recommendations:
        await ctx.send(f"Recommended artists based on '{artist_info}':")
        for idx, artist in enumerate(recommendations, 1):
            await ctx.send(f"{idx}. {artist}")
    else:
        await ctx.send("Artist not found or recommendations unavailable.")
        
@bot.command(name="albumRec", help="gets album recommendations based off inputed artist and album")
async def album_look(ctx, *, song_info):
    split_info = song_info.split(',', 1)
    if len(split_info) != 2:
        await ctx.send("Please use the format: Album name, Artist Name")
        return
    album, artist = map(str.strip, split_info)
    recommendations = get_album_recommendations(album, artist)
    await ctx.send(f"Recommended albums based on '{artist}' - '{album}':")
    for idx, album in enumerate(recommendations, 1):
        await ctx.send(f"{idx}. {album}")
bot.run(Token)

[2024-01-10 23:00:30] [INFO    ] discord.client: logging in using static token
[2024-01-10 23:00:30] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: 8e91352f146134fde9ae93b1c1e5e4db).


1172617540392992768 has connected to Discord.
Found album ID: 4SZko61aMnmgvNhfhgTuD3
